In [1]:
########## Setup ##########
###########################
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)

In [2]:
########## Load Data ##########
###############################
master = pd.read_csv('master.csv')

# Create a copy of the master dataframe to work on
working = master

In [3]:
########## Evaluate Data ##########
###################################

# View a sample of columns
print("""
master sample""")
master



master sample


,ID,age,gender,usage_meeting_place,usage_worship,usage_arts,usage_wellbeing,usage_connections,usage_model_sustainability,usage_flexible_community/church,...,usage_reflection,usage_other,modify_exterior,modify_interior,modify_sustainable_building,modify_layout,modify_cafe,activities_like_to_see,float,contribute
0,3,31 - 49,Female,Agree,Agree,Neutral,Disagree,Agree,Strongly Agree,Strongly Agree,...,Disagree,NaN,Agree,Strongly Agree,Neutral,Agree,Disagree,A toddler group;Fitness / yoga / pilates / dan...,NaN,No thanks
1,4,50 - 70,Male,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Strongly Agree,...,Agree,NaN,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Drama;Creative workshops;Arts exhibitions;Live...,NaN,YES - I would be interested in becoming a vol...
2,5,50 - 70,male,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Agree,NaN,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Drama;Photography;Creative workshops;Arts exhi...,I would love to see a community hub and arts a...,YES - I would be interested in becoming a vol...
3,6,50 - 70,Female,Strongly Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Agree,NaN,Neutral,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Drama;Creative workshops;Creative writing;Live...,NaN,YES - I would be interested in becoming a vol...
4,7,50 - 70,Male,Strongly Agree,Agree,Strongly Agree,Disagree,Disagree,Strongly disagree,Strongly Agree,...,Neutral,NaN,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Disagree,Drama;Creative workshops;Arts exhibitions;Crea...,NaN,YES - I would be interested in becoming a vol...
5,8,50 - 70,Female,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,NaN,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Live music;Arts exhibitions;Creative workshops...,NaN,YES - I would be interested in becoming a vol...
6,9,31 - 49,Female,Agree,Neutral,Agree,Agree,Agree,Agree,Neutral,...,Disagree,"A space for other pop up events, such as 'Ted ...",Agree,Agree,Strongly Agree,Agree,Agree,Live music;Live theatre;Community cafe;Communi...,Suggestions in previous box,YES - I would be interested in becoming a vol...
7,10,31 - 49,Female,Strongly Agree,Neutral,Neutral,Agree,Strongly Agree,Neutral,Neutral,...,Strongly Agree,A space for young people to meet and interact....,Agree,Neutral,Neutral,Neutral,Strongly Agree,Community cafe;Community coffee mornings;Couns...,A space for families to meet and share experie...,No thanks
8,11,31 - 49,male,Strongly Agree,Strongly disagree,Agree,Agree,Strongly Agree,Agree,Neutral,...,Neutral,NaN,Agree,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Live music;Live theatre;Community cafe;Communi...,NaN,YES - I would be interested in becoming a vol...
9,12,70 +,Male,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,...,Strongly Agree,NaN,Agree,Agree,Agree,Strongly Agree,Strongly Agree,Drama;Arts exhibitions;Live music;Music / sing...,NaN,No thanks


In [4]:
# View summary of values
print("""
master summary
""", master.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 22 columns):
ID                                 202 non-null int64
age                                202 non-null object
gender                             188 non-null object
usage_meeting_place                202 non-null object
usage_worship                      202 non-null object
usage_arts                         202 non-null object
usage_wellbeing                    202 non-null object
usage_connections                  202 non-null object
usage_model_sustainability         202 non-null object
usage_flexible_community/church    202 non-null object
usage_services                     202 non-null object
usage_festivals                    202 non-null object
usage_reflection                   202 non-null object
usage_other                        41 non-null object
modify_exterior                    202 non-null object
modify_interior                    202 non-null object
modify_sustainabl

In [5]:
# 202 replies
# All but ID are objects (convert data to numerical)
# 188 gender (impute missing replies)
# 41 usage_other (don't fill but extract data)
# 79 float (don't fill but extract data)
# 176 contribute (don't fill but extract data)

In [6]:
# usage_other, activities_like_to_see, float, contribute all feature string data (not optional). 

# Create a working dataframe without these columns
working_short = working.drop(["ID", "usage_other", "activities_like_to_see", "float", "contribute"], axis=1)

# Check
working_short.head()

,age,gender,usage_meeting_place,usage_worship,usage_arts,usage_wellbeing,usage_connections,usage_model_sustainability,usage_flexible_community/church,usage_services,usage_festivals,usage_reflection,modify_exterior,modify_interior,modify_sustainable_building,modify_layout,modify_cafe
0,31 - 49,Female,Agree,Agree,Neutral,Disagree,Agree,Strongly Agree,Strongly Agree,Neutral,Agree,Disagree,Agree,Strongly Agree,Neutral,Agree,Disagree
1,50 - 70,Male,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree
2,50 - 70,male,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree
3,50 - 70,Female,Strongly Agree,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Neutral,Neutral,Agree,Neutral,Agree,Strongly Agree,Strongly Agree,Strongly Agree
4,50 - 70,Male,Strongly Agree,Agree,Strongly Agree,Disagree,Disagree,Strongly disagree,Strongly Agree,Neutral,Neutral,Neutral,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Agree,Strongly Disagree


In [7]:
########## Clean the data ##########
####################################

In [8]:
# Age

# View a tally of values
print("""
age sample
""", working_short["age"].value_counts())


age sample
 50 - 70    87
31 - 49    66
70 +       29
18 - 30    20
Name: age, dtype: int64


In [9]:
# Convert values to integers
working_short["age"].replace(["18 - 30", "31 - 49", "50 - 70", "70 +"], [0, 1, 2, 3], inplace=True)

# Check
print("""
modified age sample
""", working_short["age"].value_counts())

print("""
Train Data summary
""", working_short.info())


modified age sample
 2    87
1    66
3    29
0    20
Name: age, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 17 columns):
age                                202 non-null int64
gender                             188 non-null object
usage_meeting_place                202 non-null object
usage_worship                      202 non-null object
usage_arts                         202 non-null object
usage_wellbeing                    202 non-null object
usage_connections                  202 non-null object
usage_model_sustainability         202 non-null object
usage_flexible_community/church    202 non-null object
usage_services                     202 non-null object
usage_festivals                    202 non-null object
usage_reflection                   202 non-null object
modify_exterior                    202 non-null object
modify_interior                    202 non-null object
modify_sustainable_building        202 non-null 

In [10]:
# Gender

# View a tally of values
print("""
gender sample
""", working_short["gender"].value_counts())


gender sample
 Female             85
Male               50
Female             27
male               10
female              3
M                   3
female              2
Woman               1
1 Male 1 Female     1
F                   1
Femalw              1
Woman               1
Male                1
Can't remember      1
Neutral             1
Name: gender, dtype: int64


In [11]:
# Map values to summarised version (male, female and NB)
working_short["gender"] = working_short["gender"].map({"Female":"female", "Male":"male", "Female ":"female" ,"male":"male", "M":"male", "female":"female", "female ":"female", "Can't remember":"NB", "1 Male 1 Female":"male", "1 Male 1 Female":"female", "Woman":"female", "Male ":"male", "F":"female", "Femalw":"female", "Neutral":"NB", "Woman ":"female"})

# Check mapping
print("""
gender sample mapped
""", working_short["gender"].value_counts())


gender sample mapped
 female    122
male       64
NB          2
Name: gender, dtype: int64


In [12]:
# Convert values to integers
working_short["gender"].replace(["NB", "male", "female"], [0, 1, 2], inplace=True)

# Check
print("""
modified gender sample
""", working_short["gender"].value_counts())

print("""
Train Data summary
""", working_short.info())

# Converted to float? 


modified gender sample
 2.0    122
1.0     64
0.0      2
Name: gender, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 17 columns):
age                                202 non-null int64
gender                             188 non-null float64
usage_meeting_place                202 non-null object
usage_worship                      202 non-null object
usage_arts                         202 non-null object
usage_wellbeing                    202 non-null object
usage_connections                  202 non-null object
usage_model_sustainability         202 non-null object
usage_flexible_community/church    202 non-null object
usage_services                     202 non-null object
usage_festivals                    202 non-null object
usage_reflection                   202 non-null object
modify_exterior                    202 non-null object
modify_interior                    202 non-null object
modify_sustainable_building        202 n

In [13]:
# Usage

# View a sample of values
print("""
usage sample
""", working_short["usage_worship"].value_counts())


usage sample
 Neutral              72
Strongly Agree       61
Agree                51
Disagree             12
Strongly disagree     6
Name: usage_worship, dtype: int64


In [14]:
working_short.replace(["Strongly Disagree", "Disagree", "Neutral", "Agree", "Strongly Agree"], [0, 1, 2, 3, 4], inplace=True)

# Check
print("""
modified usage sample
""", working_short["usage_worship"].value_counts())

print("""
Train Data summary
""", working_short.info())


modified usage sample
 2                    72
4                    61
3                    51
1                    12
Strongly disagree     6
Name: usage_worship, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 17 columns):
age                                202 non-null int64
gender                             188 non-null float64
usage_meeting_place                202 non-null int64
usage_worship                      202 non-null object
usage_arts                         202 non-null object
usage_wellbeing                    202 non-null int64
usage_connections                  202 non-null int64
usage_model_sustainability         202 non-null object
usage_flexible_community/church    202 non-null object
usage_services                     202 non-null int64
usage_festivals                    202 non-null int64
usage_reflection                   202 non-null int64
modify_exterior                    202 non-null int64
modify_in

In [15]:
# worship, arts, model_sustainability, flexible_community/church still have objects

# View a sample of the dataframe
working_short

,age,gender,usage_meeting_place,usage_worship,usage_arts,usage_wellbeing,usage_connections,usage_model_sustainability,usage_flexible_community/church,usage_services,usage_festivals,usage_reflection,modify_exterior,modify_interior,modify_sustainable_building,modify_layout,modify_cafe
0,1,2.0,3,3,2,1,3,4,4,2,3,1,3,4,2,3,1
1,2,1.0,4,4,4,4,4,3,4,4,4,3,4,4,4,4,4
2,2,1.0,4,3,4,4,4,4,4,4,4,3,4,4,4,4,4
3,2,2.0,4,2,4,4,4,4,4,2,2,3,2,3,4,4,4
4,2,1.0,4,3,4,1,1,Strongly disagree,4,2,2,2,4,4,4,4,0
5,2,2.0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
6,1,2.0,3,2,3,3,3,3,2,2,1,1,3,3,4,3,3
7,1,2.0,4,2,2,3,4,2,2,2,2,4,3,2,2,2,4
8,1,1.0,4,Strongly disagree,3,3,4,3,2,2,3,2,3,4,3,4,4
9,3,1.0,4,4,4,4,4,4,4,4,4,4,3,3,3,4,4


In [16]:
# "Strongly Disagree" also features as "Strongly disagree"

working_short.replace("Strongly disagree", 0, inplace=True)

# Check
print("""
Train Data summary
""", working_short.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 17 columns):
age                                202 non-null int64
gender                             188 non-null float64
usage_meeting_place                202 non-null int64
usage_worship                      202 non-null int64
usage_arts                         202 non-null int64
usage_wellbeing                    202 non-null int64
usage_connections                  202 non-null int64
usage_model_sustainability         202 non-null int64
usage_flexible_community/church    202 non-null int64
usage_services                     202 non-null int64
usage_festivals                    202 non-null int64
usage_reflection                   202 non-null int64
modify_exterior                    202 non-null int64
modify_interior                    202 non-null int64
modify_sustainable_building        202 non-null int64
modify_layout                      202 non-null int64
modify_cafe                  

In [17]:
# Encode age using OneHotEncoder
age_encoded = pd.DataFrame(encoder.fit_transform(working_short[["age"]]))

# Join to working_short dataframe
working_short_encoded = working_short.join(age_encoded)

# Drop age from working_short
working_short_encoded = working_short_encoded.drop("age", axis=1)

# Check 
working_short_encoded


,gender,usage_meeting_place,usage_worship,usage_arts,usage_wellbeing,usage_connections,usage_model_sustainability,usage_flexible_community/church,usage_services,usage_festivals,usage_reflection,modify_exterior,modify_interior,modify_sustainable_building,modify_layout,modify_cafe,0,1,2,3
0,2.0,3,3,2,1,3,4,4,2,3,1,3,4,2,3,1,0.0,1.0,0.0,0.0
1,1.0,4,4,4,4,4,3,4,4,4,3,4,4,4,4,4,0.0,0.0,1.0,0.0
2,1.0,4,3,4,4,4,4,4,4,4,3,4,4,4,4,4,0.0,0.0,1.0,0.0
3,2.0,4,2,4,4,4,4,4,2,2,3,2,3,4,4,4,0.0,0.0,1.0,0.0
4,1.0,4,3,4,1,1,0,4,2,2,2,4,4,4,4,0,0.0,0.0,1.0,0.0
5,2.0,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,0.0,0.0,1.0,0.0
6,2.0,3,2,3,3,3,3,2,2,1,1,3,3,4,3,3,0.0,1.0,0.0,0.0
7,2.0,4,2,2,3,4,2,2,2,2,4,3,2,2,2,4,0.0,1.0,0.0,0.0
8,1.0,4,0,3,3,4,3,2,2,3,2,3,4,3,4,4,0.0,1.0,0.0,0.0
9,1.0,4,4,4,4,4,4,4,4,4,4,3,3,3,4,4,0.0,0.0,0.0,1.0


In [18]:
########## Analysis ##########
##############################

In [19]:
corr_matrix = working_short_encoded.corr()

In [20]:
corr_matrix[2].sort_values(ascending=False)

2                                  1.000000
modify_interior                    0.153928
usage_meeting_place                0.153663
usage_flexible_community/church    0.137390
modify_sustainable_building        0.108252
usage_model_sustainability         0.088512
usage_connections                  0.077914
usage_wellbeing                    0.051112
modify_layout                      0.040095
usage_arts                         0.039195
modify_cafe                        0.025598
modify_exterior                    0.014073
usage_reflection                   0.004800
gender                            -0.001368
usage_worship                     -0.001656
usage_services                    -0.004404
usage_festivals                   -0.069713
0                                 -0.288330
3                                 -0.356112
1                                 -0.605917
Name: 2, dtype: float64